# Detect claims to fact check in political debates

In this project you will implement various classifiers using both neural and feature based technqiues to detect which sentences in political debates should be fact checked.
Dataset from ClaimBuster: https://zenodo.org/record/3609356 
Evaluate your classifiers using the same metrics as http://ranger.uta.edu/~cli/pubs/2017/claimbuster-kdd17-hassan.pdf (Table 2)

Classification report from sklearn provides everything

In [1]:
# TODO:  Create advanced model(s) (suggestions are given below)
#           -- Generate more features that a model can use. For example the context around the sentence, sentiment, named entities etc.
#           -- Rule based classifier. For example, if sentence contains certain words, tags, statistics etc.
#           -- Deep learning (word embeddings, transformer models etc.)
#           -- Sub-sentence classifier. Long sentences may include several claims, so the goal is to mark the span of claim(s) within a sentence

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import collections
import string

from sklearn.cluster import KMeans
from sklearn.metrics import *
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier

import json
import glob
import re
import time 

# Loading and merging the data

In [3]:
file1 = pd.read_csv("data/crowdsourced.csv", encoding='utf-8')
file2 = pd.read_csv("data/groundtruth.csv", encoding='utf-8')
df = pd.concat([file1, file2])


df["date"] = df["File_id"].str.strip(to_strip=".txt")

df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace= True)
df["mos_before_election"] = 11 - df["date"].dt.month

df['index'] = pd.RangeIndex(len(df))
df.set_index('index', inplace=True)
df


,Sentence_id,Text,Speaker,Speaker_title,Speaker_party,File_id,Length,Line_number,Sentiment,Verdict,date,mos_before_election
index,,,,,,,,,,,,
0,8211,"Now, this is not standing still.",Richard M. Nixon,Vice President,REPUBLICAN,1960-09-26.txt,6,114,-0.417840,-1,1960-09-26,2
1,8515,So these are three programs which are quite mo...,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,9,418,0.249581,-1,1960-09-26,2
2,8514,The proposal advanced by you and by Mr. Javits...,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,42,417,-0.626563,1,1960-09-26,2
3,8513,It does not put a deficit on the Treasury.,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,9,416,-0.629486,1,1960-09-26,2
4,8512,The third is medical care for the aged which i...,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,22,415,0.000000,-1,1960-09-26,2
...,...,...,...,...,...,...,...,...,...,...,...,...
23528,34028,"First of all, the media is so dishonest and so...",Donald Trump,Businessman,REPUBLICAN,2016-10-19.txt,17,907,0.032300,-1,2016-10-19,1
23529,34027,What I've seen -- what I've seen is so bad.,Donald Trump,Businessman,REPUBLICAN,2016-10-19.txt,9,906,-0.669600,-1,2016-10-19,1
23530,34026,I'll look at it at the time.,Donald Trump,Businessman,REPUBLICAN,2016-10-19.txt,7,905,0.000000,-1,2016-10-19,1


# Data preprocessing

In [4]:
def remove_punctuation(text):
    tokens = re.sub('[^a-zA-Z]', ' ', text).lower()
    return tokens

In [5]:
def remove_stop_words(text):
    stop_words = stopwords.words('english')
    word_list = [word for word in text.split() if word not in stop_words]
    return word_list

In [6]:
def get_word_stemm(word_list):
    """Stemmers remove morphological affixes from words, leaving only the word stem."""
    stemmer = SnowballStemmer('english')
    singles = [stemmer.stem(word) for word in word_list] 
    return singles

In [7]:
def preprocess_data(docs, stemming=False):

    text_list = [] 
    for doc in docs:  
        # 1. Remove punctuation and set as lower case
        text = remove_punctuation(doc)

        # 2. Remove stop words and extra spaces
        word_list = remove_stop_words(text)

        if stemming:
            # 3. Stemming
            word_list = get_word_stemm(word_list)

        joined_text = " ".join(word_list)
        text_list.append(joined_text)
        
    return text_list



In [8]:
data = preprocess_data(df.Text.values, stemming=True)

In [9]:
data

['stand still',
 'three program quit moder',
 'propos advanc mr javit would cost six hundr million dollar mr rockefel reject new york said agre financ said ought social secur',
 'put deficit treasuri',
 'third medic care age tie social secur financ social secur fund',
 'say major republican oppos',
 'say major',
 'say democrat unit support program',
 'extrem bill yet could get one republican join least think four eight democrat vote send floor hous one republican join democrat oppos',
 'fact matter bill less recommend mr nixon morn propos',
 'uh defeat teacher salari bill uh met opinion need',
 'second feder aid educ bill',
 'think extrem yet near two third three fourth republican hous repres vote propos',
 'one bill dollar twenti five cent hour anyon work store compani million dollar year busi',
 'well let look bill vice presid suggest extrem',
 'reason presid eisenhow veto sustain reason congress send bill think veto peopl congress major know countri behind presid',
 'must alway reme

In [10]:
# add clean text to dataframe
df["Clean_text"] = pd.Series(data)

In [11]:
df.head()

,Sentence_id,Text,Speaker,Speaker_title,Speaker_party,File_id,Length,Line_number,Sentiment,Verdict,date,mos_before_election,Clean_text
index,,,,,,,,,,,,,
0,8211,"Now, this is not standing still.",Richard M. Nixon,Vice President,REPUBLICAN,1960-09-26.txt,6,114,-0.417840,-1,1960-09-26,2,stand still
1,8515,So these are three programs which are quite mo...,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,9,418,0.249581,-1,1960-09-26,2,three program quit moder
2,8514,The proposal advanced by you and by Mr. Javits...,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,42,417,-0.626563,1,1960-09-26,2,propos advanc mr javit would cost six hundr mi...
3,8513,It does not put a deficit on the Treasury.,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,9,416,-0.629486,1,1960-09-26,2,put deficit treasuri
4,8512,The third is medical care for the aged which i...,John F. Kennedy,Senator,DEMOCRAT,1960-09-26.txt,22,415,0.000000,-1,1960-09-26,2,third medic care age tie social secur financ s...


In [12]:
mask = df["date"].dt.year < 2012

X_train = df.loc[mask, "Clean_text"].values
y_train = df.loc[mask, "Verdict"].values

X_test = df.loc[~mask, "Clean_text"].values
y_test = df.loc[~mask, "Verdict"].values

# Word Embedding using keras - NOT WORKING YET :D 

In [13]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [14]:
# defining vocabulary
vocabulary = {}
sentences_len = []
for sentence in X_train:
    for term in sentence.split():
        vocabulary.setdefault(term, len(vocabulary))

In [15]:
# Defining vocabulary size
vocabulary_size = list(vocabulary.values())[-1] + 1

print(f"vocabulary is composed of {vocabulary_size} unique words")

vocabulary is composed of 6388 unique words


## One hot encoding representation

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer

Encodding train data

In [37]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
encoded_docs = tokenizer.texts_to_sequences(X_train)

Encodding Test data

In [46]:
encoded_test = tokenizer.texts_to_sequences(X_test)

## Padding sequences

In [42]:
# finding max sentence length

vec_lengths = []
for i in encoded_docs:
    vec_lengths.append(len(i))


max_length = np.unique(vec_lengths)[-1]


In [43]:
embedded_docs=pad_sequences(encoded_docs,padding='post',maxlen=max_length)
print(embedded_docs)

[[ 190  179    0 ...    0    0    0]
 [ 163   41  658 ...    0    0    0]
 [ 113  933   37 ...    0    0    0]
 ...
 [   2 2635   19 ...    0    0    0]
 [ 135   60  608 ...    0    0    0]
 [4353  230    3 ...    0    0    0]]


For training

In [44]:
embedded_docs.shape

(18170, 65)

For testing

In [47]:
embedded_test=pad_sequences(encoded_test,padding='post',maxlen=max_length)
print(embedded_test.shape)

(5363, 65)


## Creating the model

In [55]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D
import tensorflow.keras.backend

In [ ]:
tensorflow.keras.backend.clear_session()


In [ ]:
labels = df['Verdict'].copy()

In [ ]:
labels = labels.apply(lambda x: 1 if x==1 else 0)

In [ ]:
# Preparing for training the model
X_embedded = np.array(embedded_docs)
y = labels
#y = df['Verdict'].values

# check shapes
print(f"X shape: {X_embedded.shape}. y shape: {y.shape}")

In [ ]:
# Train test split
X_embedded_train = X_embedded[:sum(mask)]
y_embedded_train = y[:sum(mask)]

X_embedded_test = X_embedded[sum(mask):]
y_embedded_test = y[sum(mask):]

print('Train shape: ', (X_embedded_train.shape, y_embedded_train.shape))
print('Test shape: ', (X_embedded_test.shape, y_embedded_test.shape))


In [ ]:
tensorflow.keras.backend.clear_session()

In [ ]:
model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size, 100, input_length=max_length))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(100, 8, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=10))
model_conv.add(LSTM(100))

model_conv.add(Dense(1, activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_conv.summary()

In [ ]:
model_conv.fit(X_embedded_train,y_embedded_train, validation_split=0.4, epochs = 10)

## Encoding the labels for multiclass problem


In [63]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1))

y_encoded.shape

(18170, 3)

In [64]:
embedded_docs.shape

(18170, 65)

In [65]:
y_encoded_test = one_hot_encoder.transform(y_test.reshape(-1,1))
y_encoded_test.shape

(5363, 3)

In [67]:
type(y_encoded)

numpy.ndarray

In [53]:
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D
import tensorflow
tensorflow.keras.backend.clear_session()



In [56]:
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocabulary_size, embedding_dim, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 65, 100)           638800    
                                                                 
 conv1d_1 (Conv1D)           (None, 61, 128)           64128     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
Total params: 704,251
Trainable params: 704,251
Non-trainable params: 0
________________________________________________

In [60]:
model.fit(embedded_docs,,validation_data=0.2,epochs=10)

Epoch 1/10
 72/568 [==>...........................] - ETA: 4s - loss: 0.8714 - accuracy: 0.6593

InvalidArgumentError:  indices[6,17] = 6388 is not in [0, 6388)
	 [[node sequential_1/embedding_1/embedding_lookup
 (defined at C:\Users\bruna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\layers\embeddings.py:191)
]] [Op:__inference_train_function_921]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_1/embedding_1/embedding_lookup:
In[0] sequential_1/embedding_1/embedding_lookup/641:	
In[1] sequential_1/embedding_1/Cast (defined at C:\Users\bruna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\layers\embeddings.py:190)

Operation defined at: (most recent call last)
>>>   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3312.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>